In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [1]:
import psycopg2 as ps
import pandas as pd

conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()
q = '''SELECT * FROM TWEETS
        WHERE TWEET_CREATED >= '2018-01-01' 
        AND TWEET_CREATED < '2021-01-01'; '''
cursor.execute(q)
results = cursor.fetchall()
colnames = [desc[0] 
        for desc in cursor.description]
cursor.close()
conn.close()

df = pd.DataFrame(results, columns=colnames)

In [2]:
import Preprocessing
import time

start = time.time()
df_clean = Preprocessing.clean_text(df, 'tweet_text')
end = time.time()
print(str(end-start), 'ms')

15.059785604476929 ms


In [3]:
df_clean.clean_text = df_clean.clean.copy()
df_clean = df_clean.drop('clean', axis=1)

In [4]:
# shift column 'Name' to first position
clean_text = df_clean.pop('clean_text') 
# insert column using insert(position,column_name,
# first_column) function
df_clean.insert(6, 'clean_text', clean_text)

In [8]:
df_clean.head(100)

,tweet_id,user_id,user_name,tweet_text,tweet_created,tweet_link,clean_text,isincident,identified_place,place_id,isaccident,isobstacle,isdanger,incident_desc
0,1013816364220518402,@VOSTpanama,VOST Panamá,"PRECAUCION\ncALLE 50 anegada, Vía \n@TraficoC...",2018-07-02 11:07:11-05:00,https://twitter.com/VOSTpanama/status/10138163...,precaucion calle 50 anegada via,None,None,None,None,None,None,None
1,951132392906870784,@e61a52841a184cf,Yan,@TReporta\n @tvnnoticias\n @nexnoticias\n @Tra...,2018-01-10 11:43:28-05:00,https://twitter.com/e61a52841a184cf/status/951...,recorrido mañanitas,None,None,None,None,None,None,None
2,1010612149826342914,@TReporta,Telemetro Reporta,Inundada se encuentra Calle 70 en San Fransisc...,2018-06-23 14:54:47-05:00,https://twitter.com/TReporta/status/1010612149...,inundada calle 70 san fransisco,None,None,None,None,None,None,None
3,1013816934142562304,@TraficoCPanama,Tráfico Panamá,Inundada la Calle 74 San Francisco precaución...,2018-07-02 11:09:27-05:00,https://twitter.com/TraficoCPanama/status/1013...,inundada calle 74 san francisco precaucion ruta,None,None,None,None,None,None,None
4,1022561878969671680,@O_YSierra,YOS,Inundada la avenida frente al mercado del mari...,2018-07-26 14:18:44-05:00,https://twitter.com/O_YSierra/status/102256187...,inundada avenida frente mercado marisco,None,None,None,None,None,None,None
5,1025843029645688832,@grupoelite507,@grupoelite507,Situación en Nuevo Arraijan en este momento.. ...,2018-08-04 15:36:52-05:00,https://twitter.com/grupoelite507/status/10258...,situacion arraijan,None,None,None,None,None,None,None
6,1046809997320093697,@TReporta,Telemetro Reporta,.\n@TraficoCPanama\n informa cierre del Corred...,2018-10-01 12:12:06-05:00,https://twitter.com/TReporta/status/1046809997...,informa cierre corredor norte altura villa lucre,None,None,None,None,None,None,None
7,947703346781093888,@TraficoCPanama,Tráfico Panamá,"""Feliz año 2018"" En este nuevo año, el equipo ...",2018-01-01 00:37:40-05:00,https://twitter.com/TraficoCPanama/status/9477...,feliz año 2018 año equipo trafico panama desea...,None,None,None,None,None,None,None
8,947789777079799808,@m_mijail,Mijail,Atencion \n@TraficoCPanama,2018-01-01 06:21:06-05:00,https://twitter.com/m_mijail/status/9477897770...,atencion,None,None,None,None,None,None,None
9,947832759560691712,@karlostho,Karlos,Que diferencias!! Felicidades al \n@PanamaAme...,2018-01-01 09:11:54-05:00,https://twitter.com/karlostho/status/947832759...,diferencias felicidades portada propicia dia r...,None,None,None,None,None,None,None


In [9]:
# for i in range(2014, 2021):
#     df_clean[(df_clean.tweet_created>='%i-01-01'%i) & (df_clean.tweet_created<'%i-01-01'%(i+1))].to_csv('tweets_%i.csv'%(i), encoding='utf-8')
df_clean.to_csv('tweets-2018-2020.csv', encoding='utf-8')